In [ ]:
import os

In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark Session with Default Configurations").getOrCreate()
from pyspark.sql.functions import date_format,col

### Set path to Parquet file(s) in the Iguazio Platform

In [ ]:
file_path = os.path.join(os.getenv('V3IO_HOME_URL')+'/%s'% 'data/weather/weathercsv2M.csv')
file_path

### Load Spark Dataframe from Parquet

In [25]:
%%time
df = spark.read\
    .format("csv")\
    .option("header", "true")\
    .option("inferSchema", "true")\
    .load(file_path)

CPU times: user 3.66 ms, sys: 168 µs, total: 3.83 ms
Wall time: 9.11 s


### Explore Spark Dataframe

In [26]:
%%time
df.show()

+------+-----+-------------------+------------------+-------------------+---------+---------+---------+-----------+--------------+-------------+-----------------------+--------------------+----------+-----------+---------+------------------+---------------+------------+----+---+-------+
|  usaf| wban|           datetime|          latitude|          longitude|elevation|windAngle|windSpeed|temperature|seaLvlPressure|cloudCoverage|presentWeatherIndicator|pastWeatherIndicator|precipTime|precipDepth|snowDepth|       stationName|countryOrRegion|         p_k|year|day|version|
+------+-----+-------------------+------------------+-------------------+---------+---------+---------+-----------+--------------+-------------+-----------------------+--------------------+----------+-----------+---------+------------------+---------------+------------+----+---+-------+
|999999| 4237|2020-06-13 04:00:00|47.513999999999996|-123.81200000000001|     87.0|      0.0|      0.0|       11.7|          null|      

In [ ]:
%%time
df.printSchema()

In [27]:
%%time
df_part = df.withColumn("month", date_format(col("datetime"), "MM"))

CPU times: user 1.83 ms, sys: 78 µs, total: 1.91 ms
Wall time: 6.63 ms


In [ ]:
df_part.show()

### Print Dataframe Schema
NOTE: We will use this schema to create a Hive table 

SQL file : weathercsv-parquet_external.sql

In [ ]:
%%time
df_part.printSchema()

In [ ]:
%%time
df_part.createOrReplaceTempView('weather')

In [ ]:
%%time
sqlDF = spark.sql("SELECT * FROM weather limit 10")

In [ ]:
%%time
sqlDF.show()

In [29]:
%%time
df_part.write.parquet("v3io://users/%s/data/weather_parquet/"% os.getenv('V3io_USERNAME'), mode='overwrite',partitionBy=['year','month'])

CPU times: user 1.04 ms, sys: 3.07 ms, total: 4.12 ms
Wall time: 8.01 s


In [ ]:
spark.stop()